In [1]:
# Colab settings



In [2]:
#import part 
import matplotlib.pyplot as plt
from IPython import display
from models import LinearRegressionWithGd
from common import *
import pandas as pd
path = "Dataset/Training/"

In [3]:
# configuration
num_folds = 5
epochs = 1000
learning_rate = 0.1

lrg = LinearRegressionWithGd()
lrg.learning_rate = learning_rate

df_1, df_2 = folding(path)
np.random.shuffle(df_1)


In [35]:
def make_table(stats, loss, betas, bias):
    columns = ["Val MSE", "Val R2", "Val RMSE", "Val R2/RMSE"] + [f"p{p+1}" for p in range(len(betas))] + ["bias"]
    # mean = np.mean([np.sqrt(loss)] + [stats] + [betas] + [bias])
    mean = np.mean(np.sqrt(loss) + stats + betas + bias)
    std = np.std(np.sqrt(loss) + stats + betas + bias)
    index = [f"Fold {i+1}" for i, _ in enumerate(stats) ] + ["Mean"] + ["Std"]
    df = pd.DataFrame(np.array([np.sqrt(loss)] + [stats] + [mean] + [std]),
                  columns=columns,
                  index=index)
    display.display(df)   
    
    pass

### Training

In [5]:
features = []
bias = []
stats = []
losses = []
for i in range(num_folds):
    train, test = cv_loo(df_1, num_folds, i)

    X_train = normalized(train.T[0:53]).T
    X_test = normalized(test.T[0:53]).T

    Y_train = train.T[53].T
    Y_train = np.expand_dims(Y_train, axis=-1)

    Y_test = test.T[53].T
    Y_test = np.expand_dims(Y_test, axis=-1)

    loss = lrg.train(epochs, X_train, Y_train)
    stat = lrg.validate(X_test, Y_test)
    losses.append(loss)
    stats.append(stat)
    features.append(lrg.betas)
    bias.append(lrg.bias)
    
    # graphs_plot(losses, stats[0], stats[1], stats[2])

mean_stats = np.sum(stats, axis=0) / num_folds
print("mse_val is {}, r2_val is {}, rmse is {}, r2/rmse {}".format(
    mean_stats[0], mean_stats[1], mean_stats[2], mean_stats[3]))


Loss: 983.0983: 100%|██████████| 1000/1000 [02:43<00:00,  6.13it/s]


val loss is 979.8665163265865, R2 is 0.10219568684502023, RMSE is 31.30281962262484, R2/RMSE is 0.0032647438178749857
val loss is 1153.9476014386198, R2 is 0.13838058786894736, RMSE is 33.96980425964536, R2/RMSE is 0.004073635126397753
val loss is 928.6936218318536, R2 is 0.16767266997670327, RMSE is 30.474474922988477, R2/RMSE is 0.005502069203831271
val loss is 913.1398617294907, R2 is 0.18388740204818432, RMSE is 30.21820414467893, R2/RMSE is 0.006085318676376893
val loss is 1005.8675655099954, R2 is 0.17674139827668833, RMSE is 31.71541526623915, R2/RMSE is 0.005572728491587129
mse_val is 996.3030333673092, r2_val is 0.1537755490031087, rmse is 31.536143643235352, r2/rmse 0.004899699063213607


In [36]:

make_table([list(stat) for stat in stats], losses, features, bias)

pass

ValueError: operands could not be broadcast together with shapes (5,) (5,4) 

In [72]:
stats = [list(stat) for stat in stats] 
# np.sqrt(losses)
# mean = np.mean(np.sqrt(loss) + stats + bias, axis=0)
mean = [val for val in [[np.sqrt(losses)] + stats]]
# mean = np.mean(np.sqrt(losses) + stats, axis=0)
# mean = np.mean( stats + bias, axis=0 )
print(mean)

[[array([34.14349643, 32.44462416, 32.67248738, 32.23004622, 31.49934581]), [979.8665163265865, 0.10219568684502023, 31.30281962262484, 0.0032647438178749857], [1153.9476014386198, 0.13838058786894736, 33.96980425964536, 0.004073635126397753], [928.6936218318536, 0.16767266997670327, 30.474474922988477, 0.005502069203831271], [913.1398617294907, 0.18388740204818432, 30.21820414467893, 0.006085318676376893], [1005.8675655099954, 0.17674139827668833, 31.71541526623915, 0.005572728491587129]]]


In [99]:
print([b[0][] for b in bias])
# print(np.concatenate((losses, bias), axis=1))
# print(np.sqrt(losses).T, bias)

[array([5.67543146]), array([4.52294539]), array([3.5810703]), array([2.77649903]), array([2.08669559])]


MD
